Persis Trial-5 tapi pola datanya baru

### Import Library

In [62]:
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import requests
import matplotlib.pyplot as plt
import os

### Function

In [63]:
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:(i + n_steps), 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)

def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

def calculate_rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted)**2))

def calculate_mae(actual, predicted):
    return np.mean(np.abs(actual - predicted))

In [64]:
def run_experiment(csv_file_path, test_periods, timesteps_range):
    data = pd.read_csv(csv_file_path, sep=';')
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d').dt.date
    
    results = []

    for test_start, test_end in test_periods:
        for n_steps in range(timesteps_range[0], timesteps_range[1] + 1):
            print(f"Running experiment for test period {test_start} to {test_end}, timestep: {n_steps}")
            
            test_start_date = datetime.strptime(test_start, '%Y-%m-%d').date()
            test_end_date = datetime.strptime(test_end, '%Y-%m-%d').date()
            
            train_data = data[data['date'] < test_start_date]
            lenTrain = len(train_data)
            test_data = data[lenTrain:lenTrain+15]
            # test_data = data[(data['date'] >= test_start_date) & (data['date'] <= test_end_date)]
            
            train_energies = train_data['real'].values.reshape(-1, 1)
            test_energies = test_data['real'].values.reshape(-1, 1)
            
            scaler = MinMaxScaler()
            train_data_normalized = scaler.fit_transform(train_energies)
            test_data_normalized = scaler.transform(test_energies)
            
            X_train, y_train = prepare_data(train_data_normalized, n_steps)
            X_test, y_test = prepare_data(test_data_normalized, n_steps)
            X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
            X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
            
            model = Sequential([
                LSTM(128, activation='relu', return_sequences=True, input_shape=(n_steps, 1)),
                Dropout(0.1),
                LSTM(64, activation='relu', return_sequences=True),
                Dropout(0.1),
                LSTM(16, activation='relu'),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse')
            
            history = model.fit(X_train, y_train, epochs=200, verbose=0, validation_data=(X_test, y_test))
            
            model_name = f'Eksp_T{n_steps}_{test_end}.h5'
            model.save(os.path.join('model', model_name))

            test_energies = test_energies[:-8]
            date_test = test_data['date'][:-8]
            
            predictions = []
            current_batch = train_data_normalized[-n_steps:].reshape((1, n_steps, 1))
            
            for _ in range(len(test_energies)):
                current_pred = model.predict(current_batch, verbose=0)[0]
                predictions.append(current_pred)
                current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)
            
            predictions = scaler.inverse_transform(predictions)
            predictions = np.clip(predictions, 300, None)
            
            mape = calculate_mape(test_energies, predictions)
            mae = calculate_mae(test_energies, predictions)
            rmse = calculate_rmse(test_energies, predictions)
            
            plt.figure(figsize=(12, 6))
            plt.plot(date_test, test_energies, label='Asli')
            plt.plot(date_test, predictions, label='Prediksi')
            plt.title(f'Hasil Prediksi (Timestep: {n_steps}, Periode Test: {test_start} s.d {test_end}, MAPE: {mape:.2f})')
            plt.xlabel('Tanggal')
            plt.ylabel('Energi (Wh)')
            plt.legend()
            plt.xticks(rotation=30)
            plt.tight_layout()
            plot_name = f'Eksp_T{n_steps}_{test_end}.png'
            plt.savefig(os.path.join('plot', plot_name))
            plt.close()
            
            results.append({
                'test_period': f"{test_start} to {test_end}",
                'timestep': n_steps,
                'mape': mape,
                'mae': mae,
                'rmse': rmse,
                'model_name': model_name,
                'plot_name': plot_name
            })
    
    return results

### Main

In [65]:
# Main execution
csv_file_path = 'data/real-2mei.csv'
test_periods = [
    # ('2024-02-29', '2024-03-06'),
    ('2024-03-28', '2024-04-03'),
    # ('2024-04-18', '2024-04-24')
]
timesteps_range = (6, 14)

# Ensure directories exist
os.makedirs('model', exist_ok=True)
os.makedirs('plot', exist_ok=True)

results = run_experiment(csv_file_path, test_periods, timesteps_range)

# Print results
for result in results:
    print(f"Test Period: {result['test_period']}")
    print(f"Timestep: {result['timestep']}")
    print(f"MAPE: {result['mape']:.2f}%")
    print(f"MAE: {result['mae']:.2f}")
    print(f"Model saved as: {result['model_name']}")
    print(f"Plot saved as: {result['plot_name']}")
    print("---")

# Optional: Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('Eksp Timestep.csv', index=False)
print("Results saved to experiment_results.csv")

Running experiment for test period 2024-03-28 to 2024-04-03, timestep: 6


e:\SMT 8\Trying\LSTM\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Running experiment for test period 2024-03-28 to 2024-04-03, timestep: 7


e:\SMT 8\Trying\LSTM\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Running experiment for test period 2024-03-28 to 2024-04-03, timestep: 8


...

Running experiment for test period 2024-03-28 to 2024-04-03, timestep: 13


e:\SMT 8\Trying\LSTM\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Running experiment for test period 2024-03-28 to 2024-04-03, timestep: 14


e:\SMT 8\Trying\LSTM\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test Period: 2024-03-28 to 2024-04-03
Timestep: 6
MAPE: 100.99%
MAE: 1808.61
Model saved as: Eksp_T6_2024-04-03.h5
Plot saved as: Eksp_T6_2024-04-03.png
---
Test Period: 2024-03-28 to 2024-04-03
Timestep: 7
MAPE: 16.38%
MAE: 436.80
Model saved as: Eksp_T7_2024-04-03.h5
Plot saved as: Eksp_T7_2024-04-03.png
---
Test Period: 2024-03-28 to 2024-04-03
Timestep: 8
MAPE: 96.65%
MAE: 891.01
Model saved as: Eksp_T8_2024-04-03.h5
Plot saved as: Eksp_T8_2024-04-03.png
---
Test Period: 2024-03-28 to 2024-04-03
Timestep: 9
MAPE: 16.06%
MAE: 943.72
Model saved as: Eksp_T9_2024-04-03.h5
Plot saved as: Eksp_T9_2024-04-03.png
---
Test Period: 2024-03-28 to 2024-04-03
Timestep: 10
MAPE: 93.27%
MAE: 1153.69
Model saved as: Eksp_T10_2024-04-03.h5
Plot saved as: Eksp_T10_2024-04-03.png
---
Test Period: 2024-03-28 to 2024-04-03
Timestep: 11
MAPE: 75.20%
MAE: 680.99
Model saved as: Eksp_T11_2024-04-03.h5
Plot saved as: Eksp_T11_2024-04-03.png
---
Test Period: 2024-03-28 to 2024-04-03
Timestep: 12
MAPE: 64.4

In [70]:
df = pd.read_csv('Eksp Timestep.csv')

result = df[df['test_period'] == '2024-03-28 to 2024-04-03']['mape']
result2 = df[df['test_period'] == '2024-04-18 to 2024-04-24']['mape']

# Define a lambda function to format numbers with ',' as the decimal separator
custom_format = lambda x: f'{x:.2f}'.replace('.', ',') + '%'# Apply the formatting to the filtered dataframe
formatted_mape = result.apply(custom_format)
formatted_mape2 = result2.apply(custom_format)

# Save the formatted mape values to a text file
output_path = 'mape.txt'
with open(output_path, 'w') as file:
    file.write('\n'.join(formatted_mape))
    file.write('\n')
    file.write('\n'.join(formatted_mape2))

In [69]:
df = pd.read_csv('Eksp Timestep.csv')

result = df[df['test_period'] == '2024-03-28 to 2024-04-03']['rmse']
result2 = df[df['test_period'] == '2024-04-18 to 2024-04-24']['rmse']

# Define a lambda function to format numbers with ',' as the decimal separator
custom_format = lambda x: f'{x:.2f}'.replace('.', ',')# Apply the formatting to the filtered dataframe
formatted_mape = result.apply(custom_format)
formatted_mape2 = result2.apply(custom_format)

# Save the formatted mape values to a text file
output_path = 'rmse.txt'
with open(output_path, 'w') as file:
    file.write('\n'.join(formatted_mape))
    # file.write('\n')
    # file.write('\n'.join(formatted_mape2))